In [12]:
#%pip install scikit-learn
#%pip install matplotlib

In [2]:
import sys
import os
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Adiciona a pasta mini_project_1 como root
sys.path.append(os.path.abspath('..')) 

In [4]:
from src.models.cnf import CNF
from src.models.vector_field import VectorField

# Milestone 2: Continuous Normalizing Flow (CNF)

## Fundamentação Teórica 

### Mudança de variável

Para uma transformação invertível $z=f(x)$:

$$logp(x) = log p (z) + log \Bigl |det \frac{\partial f}{\partial x} \Bigr|$$

Para CNF com $\phi_t(x)$ integrando $\frac{dx}{dt} = f(x,t)$:

$$logp(x) = log p (z) + \int_{0}^{1}tr \Bigl( \frac{\partial f}{\partial x} \Bigr) dt$$

onde $z=\phi_0(x)$ (integração reversa)

### Propriedades da Equação de Divergência:
Para calcular a variação da log-probabilidade em um CNF, utilizamos a divergência:
 $$\text{div}(f) = \text{tr}\left( \frac{\partial f}{\partial x} \right)$$

* $f$: Campo de vetores (Vector Field) definido pela rede neural, onde $f: \mathbb{R}^d \to \mathbb{R}^d$.
* $x$: Posição no espaço de estados, $x \in \mathbb{R}^d$.
* $\frac{\partial f}{\partial x}$: Matriz Jacobiana de $f$ em relação a $x$, de dimensão $d \times d$.
* $\text{tr}(\cdot)$: Operador traço, que soma os elementos da diagonal principal da matriz Jacobiana.
* Complexidade: $O(d^2)$, já que requer $d$ passagens de autograd para extrair cada componente da diagonal.

## Treino com Moons e Circles

In [10]:
import sklearn.datasets
from sklearn.datasets import make_moons, make_circles

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dados
X, _ = make_moons(n_samples=2000, noise=0.05)
X = torch.tensor(X).float().to(device)

lr = 1e-3
batch_size = 128
n_epochs = 100

# 2. Instanciação do Modelo e Otimizador
# 'model' é a instância de CNF definida anteriormente
model = CNF(VectorField(features=2)).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)

for epoch in range(501):
    optimizer.zero_grad()
    loss = -model.log_prob(X).mean()
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")
        

def plot_density(model):
    with torch.no_grad():
        x = np.linspace(-2, 3, 100)
        y = np.linspace(-1, 2, 100)
        X, Y = np.meshgrid(x, y)
        grid = torch.tensor(np.stack([X.flatten(), Y.flatten()], axis=1)).float().to(device)
        log_prob = model.log_prob(grid).cpu().numpy()
        Z = np.exp(log_prob).reshape(100, 100)
        plt.contourf(X, Y, Z, levels=50)
        plt.title("Densidade Aprendida pelo CNF (Traço Exato)")
        plt.show()

plot_density(model)

ValueError: func must be an instance of nn.Module to specify the adjoint parameters; alternatively they can be specified explicitly via the `adjoint_params` argument. If there are no parameters then it is allowable to set `adjoint_params=()`.

## Treino com MNIST